In [11]:
from __future__ import division, print_function, absolute_import
#from Time import *
import os
import sys
import scipy.io
from itertools import groupby
import numpy as np
#from dnacgr import *
#from helper import *
####################
import warnings
#### Numba GPU + CPU Computing ####
import numba as nb
from numba import jit
from numba import njit
from numba import vectorize, float64
from numba import types
from numba.extending import overload_method

from pylab import *
from ripser import ripser, Rips
from persim import plot_diagrams
from matplotlib import pyplot as plt

from gudhi.wasserstein import wasserstein_distance
import cython 
import ot

In [42]:
from gudhi.hera import wasserstein_distance as wasserstein_distance_approx

In [12]:
import csv

In [13]:
@overload_method(types.Array, 'repeat')
def extractLabels(seqs):
    Labels = []
    append = Labels.append
    n = len(seqs)
    for i in range(n):
        label_sq = str(seqs[i].id)
        append(label_sq)
    return  Labels

In [14]:
## function to get the diagrams with given input: 
"""Input: datalist"""
@overload_method(types.Array, 'repeat')
def DiagListComp(input):
    rips = Rips(maxdim=1, coeff=2)
    DiagramsList = [rips.fit_transform(data) for data in input]
    return DiagramsList

In [31]:
# function to get the list of diagrams with order given 
@overload_method(types.Array, 'repeat')
def Diagrams_Type(Diags, order,size):
    Diags_order = [Diags[i][order] for i in range(size)]  ## Get the diagrams
    return Diags_order

In [77]:
## Function to compute the Bottle Neck distance matrix from a list of diagrams
@overload_method(types.Array, 'repeat')
def MatrixDistance(diagsList, nsize, epsilon):
    Dist = np.zeros((nsize,nsize))
    slicing = nsize
    for i in range(len(diagsList)-1):
        for j in range((i+1),slicing):
            diagi = diagsList[i]
            diagj = diagsList[j]
            Dist[i,j] = wasserstein_distance_approx(diagi, diagj,order=1.,internal_p=np.inf,delta=epsilon)
        MatDist = Dist + Dist.transpose()
    return MatDist

In [17]:
## function to read in csv matrix and convert into numpy array ---> store each in a list. 
@overload_method(types.Array, 'repeat')
def DataMatrix(filelist):
    filelist.sort(key=lambda f: int(re.sub('\D', '', f)))
    nsize = len(filelist)
    Storage = []
    append = Storage.append
    for i in range(nsize):
        fname = filelist[i]
        result = np.loadtxt(open(fname), delimiter=',')
        append(result)
    return Storage

# Google Colab running section: 

In [19]:
from google.colab import drive

In [22]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 1_dna_HRV_chaos.csv to 1_dna_HRV_chaos.csv
Saving 2_dna_HRV_chaos.csv to 2_dna_HRV_chaos.csv
Saving 3_dna_HRV_chaos.csv to 3_dna_HRV_chaos.csv
Saving 4_dna_HRV_chaos.csv to 4_dna_HRV_chaos.csv
Saving 5_dna_HRV_chaos.csv to 5_dna_HRV_chaos.csv
Saving 6_dna_HRV_chaos.csv to 6_dna_HRV_chaos.csv
Saving 7_dna_HRV_chaos.csv to 7_dna_HRV_chaos.csv
Saving 8_dna_HRV_chaos.csv to 8_dna_HRV_chaos.csv
Saving 9_dna_HRV_chaos.csv to 9_dna_HRV_chaos.csv
Saving 10_dna_HRV_chaos.csv to 10_dna_HRV_chaos.csv
Saving 11_dna_HRV_chaos.csv to 11_dna_HRV_chaos.csv
Saving 12_dna_HRV_chaos.csv to 12_dna_HRV_chaos.csv
Saving 13_dna_HRV_chaos.csv to 13_dna_HRV_chaos.csv
Saving 14_dna_HRV_chaos.csv to 14_dna_HRV_chaos.csv
Saving 15_dna_HRV_chaos.csv to 15_dna_HRV_chaos.csv
Saving 16_dna_HRV_chaos.csv to 16_dna_HRV_chaos.csv
Saving 17_dna_HRV_chaos.csv to 17_dna_HRV_chaos.csv
Saving 18_dna_HRV_chaos.csv to 18_dna_HRV_chaos.csv
Saving 19_dna_HRV_chaos.csv to 19_dna_HRV_chaos.csv
Saving 20_dna_HRV_chaos.csv to

In [25]:
FILES = os.listdir()
fileslist = [] 
for file in FILES: 
    if file.endswith('.csv'):
       fileslist.append(file)

In [26]:
DATA = DataMatrix(fileslist)

* Compute the diagrams

In [27]:
Diagrams = DiagListComp(DATA)

Rips(maxdim=1, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)


In [39]:
np.savez('DiagramsList.npz', Diagrams)

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [40]:
files.download("DiagramsList.npz")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
size = len(Diagrams)
print(size)

116


* Extracting diagrams order

In [37]:
Diags_H0 = Diagrams_Type(Diagrams, 0, size)
Diags_H1 = Diagrams_Type(Diagrams, 1, size)

In [61]:
!mkdir DiagramsH0

In [65]:
%cd DiagramsH0

/content/DiagramsH0


In [66]:
for i in range(size):
  DiagH0 = Diags_H0[i]
  title = str(i+1) + '_H0_' + '_persim_HRV' + '.txt'
  np.savetxt(title, DiagH0, delimiter = ",")

In [69]:
os.chdir('/content')

In [71]:
!mkdir DiagramsH1
%cd DiagramsH1

/content/DiagramsH1


In [72]:
for i in range(size):
  DiagH1 = Diags_H1[i]
  title = str(i+1) + '_H1_' + '_persim_HRV' + '.txt'
  np.savetxt(title, DiagH1, delimiter = ',')

In [74]:
!zip -r /content/DiagramsH0.zip /content/DiagramsH0

  adding: content/DiagramsH0/ (stored 0%)
  adding: content/DiagramsH0/99_H0__persim_HRV.txt (deflated 78%)
  adding: content/DiagramsH0/39_H0__persim_HRV.txt (deflated 78%)
  adding: content/DiagramsH0/10_H0__persim_HRV.txt (deflated 78%)
  adding: content/DiagramsH0/103_H0__persim_HRV.txt (deflated 78%)
  adding: content/DiagramsH0/96_H0__persim_HRV.txt (deflated 78%)
  adding: content/DiagramsH0/43_H0__persim_HRV.txt (deflated 78%)
  adding: content/DiagramsH0/104_H0__persim_HRV.txt (deflated 78%)
  adding: content/DiagramsH0/2_H0__persim_HRV.txt (deflated 78%)
  adding: content/DiagramsH0/55_H0__persim_HRV.txt (deflated 78%)
  adding: content/DiagramsH0/113_H0__persim_HRV.txt (deflated 78%)
  adding: content/DiagramsH0/14_H0__persim_HRV.txt (deflated 78%)
  adding: content/DiagramsH0/72_H0__persim_HRV.txt (deflated 78%)
  adding: content/DiagramsH0/45_H0__persim_HRV.txt (deflated 78%)
  adding: content/DiagramsH0/77_H0__persim_HRV.txt (deflated 78%)
  adding: content/DiagramsH0/108

In [75]:
!zip -r /content/DiagramsH1.zip /content/DiagramsH1

  adding: content/DiagramsH1/ (stored 0%)
  adding: content/DiagramsH1/16_H1__persim_HRV.txt (deflated 58%)
  adding: content/DiagramsH1/24_H1__persim_HRV.txt (deflated 58%)
  adding: content/DiagramsH1/96_H1__persim_HRV.txt (deflated 58%)
  adding: content/DiagramsH1/44_H1__persim_HRV.txt (deflated 58%)
  adding: content/DiagramsH1/40_H1__persim_HRV.txt (deflated 58%)
  adding: content/DiagramsH1/74_H1__persim_HRV.txt (deflated 58%)
  adding: content/DiagramsH1/102_H1__persim_HRV.txt (deflated 58%)
  adding: content/DiagramsH1/70_H1__persim_HRV.txt (deflated 58%)
  adding: content/DiagramsH1/115_H1__persim_HRV.txt (deflated 58%)
  adding: content/DiagramsH1/114_H1__persim_HRV.txt (deflated 58%)
  adding: content/DiagramsH1/35_H1__persim_HRV.txt (deflated 58%)
  adding: content/DiagramsH1/13_H1__persim_HRV.txt (deflated 58%)
  adding: content/DiagramsH1/51_H1__persim_HRV.txt (deflated 58%)
  adding: content/DiagramsH1/42_H1__persim_HRV.txt (deflated 58%)
  adding: content/DiagramsH1/10

In [76]:
files.download("/content/DiagramsH0.zip")
files.download("/content/DiagramsH1.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>